In [2]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

In [3]:
scope.dis()
target.dis()
exit()

In [4]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "masked-decode", env)
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [0 for _ in range(64)]
        p_temp = p0
        k_temp = k0
    elif op == "ctrl_mask":
        p_temp = p0
        k_temp = k0
    elif op == "degen":
        rand = [0 for _ in range(64)]
        p_temp = p1
        k_temp = k0
    elif op == "pmask":
        p_temp = p2
        k_temp = k0
    elif op == "pmask_small":
        p_temp = p3
        k_temp = k0
    elif op == "pmask_large":
        p_temp = p4
        k_temp = k0
    elif op == "kmask_small":
        p_temp = p0
        k_temp = k5
    elif op == "kmask_large":
        p_temp = p0
        k_temp = k6
    else:
        raise "ERROR"

    p_temp = p_temp.puf
    mem = xor(p_temp, s.encode(k_temp))
    posture(target,'k', get_bytes(mem), 16)
    return get_bytes(xor(p_temp, s.encode(rand)))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass
s = Scheme() # scheme execution helper

set_1 = [random.randrange(2) for _ in range(127)]
word = s.encode([random.randrange(2) for _ in range(64)])
set_2 = xor(set_1, word)

p0 = set_1

k0 = [random.randrange(2) for _ in range(64)]


p1 = p2 = set_2

p3 = list(p0)
for i in random.choices(range(127),k=2):
    p3[i] ^= 1
    
p4 = list(p0)
for i in random.choices(range(127),k=16):
    p4[i] ^= 1

k5 = list(k0)
for i in random.choices(range(64),k=2):
    k5[i] ^= 1

k6 = list(k0)
for i in random.choices(range(64),k=10):
    k6[i] ^= 1

p0 = PUF(p0)
p1 = PUF(p1)
p2 = PUF(p2)
p3 = PUF(p3)
p4 = PUF(p4)

    
rand = None
current_puf = None
prefix_name = "MASKED"
    

env = ENV()
# operation settings
env.root = "D:/Programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl", "ctrl_mask", "degen", "pmask", "pmask_small", "pmask_large", 
                               "kmask_small", "kmask_large"] # operations list , "set2", "set3"
env.subfolder = "masked_outside_2" # subfolder name
env.TRACES_NUM = 350 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking done inside the component, randomness fed into component, no PUF errors
masking is fed into the CW.
schemes are done followed image sent by itamar, but all p0 and p1 HDs are the same initially



1 - set1 vs set2 HD_1=HD_2
2 - set1 vs set1+3err
3 - set1 vs set2+3err
4 - set1 vs set w FI
5-8. same but with natural PUF error
9 - error vs no error PUF


PUF error model:
0.7 stable 10e-5 - 10e-6
0.3 unstable 10e-3 - 3*10e-2
python:

posture(target,'k', get_bytes(p_temp), 20)
return get_bytes(xor(p_temp, s.encode(rand)) + [0] + [0 for i in range(128)])

c code:

copying and masking
----- trig high
syndrome();
pompom();
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    

# 1
env.title = "Degenerated codeword, mask <= 0; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_1"
env.desc = "HD_1=HD_2, masking outside"
split_file(env, file_names, [0, 2], "group1")

# 2
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_2"
env.desc = "HD_1=HD_2, masking outside"
split_file(env, file_names, [1, 3], "group2")

# 3
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_3"
env.desc = "initially, HD_1=HD_2. added error: |f_{PUF1} - f_{PUF2}| = 2, masking outside"
split_file(env, file_names, [1, 4], "group3")

# 4
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_4"
env.desc = "initially, HD_1=HD_2. added error: |f_{PUF1} - f_{PUF2}| = 16, masking outside"
split_file(env, file_names, [1, 5], "group4")

# 5
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_5"
env.desc = "initially, HD_1=HD_2. added error: |f_{k1} - f_{k2}| = 2, masking outside"
split_file(env, file_names, [1, 6], "group5")

# 6
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_6"
env.desc = "initially, HD_1=HD_2. added error: |f_{k1} - f_{k2}| = 10, masking outside"
split_file(env, file_names, [1, 7], "group6")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())

2024-09-04 13-56
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12327 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12327 bytes
None
file0 file1 file2 file3 file4 file5 file6 done 2024-09-04 14-24


In [5]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "masked-decode", env)
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    rand = [random.randrange(2) for _ in range(64)]
    if op == "ctrl":
        rand = [0 for _ in range(64)]
        p_temp = p0
        k_temp = k0
    elif op == "ctrl_mask":
        p_temp = p0
        k_temp = k0
    elif op == "degen":
        rand = [0 for _ in range(64)]
        p_temp = p1
        k_temp = k0
    elif op == "pmask":
        p_temp = p2
        k_temp = k0
    elif op == "pmask_small":
        p_temp = p3
        k_temp = k0
    elif op == "pmask_large":
        p_temp = p4
        k_temp = k0
    elif op == "kmask_small":
        p_temp = p0
        k_temp = k5
    elif op == "kmask_large":
        p_temp = p0
        k_temp = k6
    else:
        raise "ERROR"

    p_temp = p_temp.sample(byte=False)
    mem = xor(p_temp, s.encode(k_temp))
    posture(target, 'k', get_bytes(mem), 16)
    return get_bytes(xor(p_temp, s.encode(rand)))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass
s = Scheme() # scheme execution helper

set_1 = [random.randrange(2) for _ in range(127)]
word = s.encode([random.randrange(2) for _ in range(64)])
set_2 = xor(set_1, word)

p0 = set_1

k0 = [random.randrange(2) for _ in range(64)]


p1 = p2 = set_2

p3 = list(p0)
for i in random.choices(range(127),k=2):
    p3[i] ^= 1
    
p4 = list(p0)
for i in random.choices(range(127),k=16):
    p4[i] ^= 1

k5 = list(k0)
for i in random.choices(range(64),k=2):
    k5[i] ^= 1

k6 = list(k0)
for i in random.choices(range(64),k=10):
    k6[i] ^= 1

p0 = PUF(p0, 0.4)
p1 = PUF(p1, 0.4)
p2 = PUF(p2, 0.4)
p3 = PUF(p3, 0.4)
p4 = PUF(p4, 0.4)

    
rand = None
current_puf = None
prefix_name = "MASKED"
    

env = ENV()
# operation settings
env.root = "D:/Programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["ctrl", "ctrl_mask", "degen", "pmask", "pmask_small", "pmask_large", 
                               "kmask_small", "kmask_large"] # operations list , "set2", "set3"
env.subfolder = "masked_outside_true_2" # subfolder name
env.TRACES_NUM = 350 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking done outside the component, randomness fed into component, PUF errors present.
masking is fed into the CW.
schemes are done followed image sent by itamar, but all p0 and p1 HDs are the same initially



1 - set1 vs set2 HD_1=HD_2
2 - set1 vs set1+3err
3 - set1 vs set2+3err
4 - set1 vs set w FI
5-8. same but with natural PUF error
9 - error vs no error PUF


PUF error model:
0.7 stable 10e-5 - 10e-6
0.3 unstable 10e-3 - 3*10e-2
python:

posture(target,'k', get_bytes(p_temp), 20)
return get_bytes(xor(p_temp, s.encode(rand)) + [0] + [0 for i in range(128)])

c code:

copying and masking
----- trig high
syndrome();
pompom();
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    

# 1
env.title = "Degenerated codeword, mask <= 0; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_1"
env.desc = "HD_1=HD_2. PUF is erroneous, masking outside"
split_file(env, file_names, [0, 2], "group1")

# 2
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_2"
env.desc = "HD_1=HD_2. PUF is erroneous, masking outside"
split_file(env, file_names, [1, 3], "group2")

# 3
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_3"
env.desc = "initially, HD_1=HD_2. added error: |f_{PUF1} - f_{PUF2}| = 2. PUF is erroneous, masking outside"
split_file(env, file_names, [1, 4], "group3")

# 4
env.title = "Codeword masking, mask <= $ ; \{f_{k}, f_{PUF1}\} vs \{f_{k}, f_{PUF2}\}"
env.name = prefix_name + "_4"
env.desc = "initially, HD_1=HD_2. added error: |f_{PUF1} - f_{PUF2}| = 16. PUF is erroneous, masking outside"
split_file(env, file_names, [1, 5], "group4")

# 5
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_5"
env.desc = "initially, HD_1=HD_2. added error: |f_{k1} - f_{k2}| = 2. PUF is erroneous, masking outside"
split_file(env, file_names, [1, 6], "group5")

# 6
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_6"
env.desc = "initially, HD_1=HD_2. added error: |f_{k1} - f_{k2}| = 10. PUF is erroneous, masking outside"
split_file(env, file_names, [1, 7], "group6")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())

2024-09-04 14-24
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12327 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12327 bytes
None
file0 file1 file2 file3 file4 file5 file6 done 2024-09-04 14-51
